In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense

In [2]:
batch_size= 32
training=keras.utils.image_dataset_from_directory('dataset_1',
                                              batch_size=batch_size,
                                             validation_split =0.2,
                                             subset ='training',
                                             seed=42,
                                             image_size=(256,256),)
val = keras.utils.image_dataset_from_directory('dataset_1',
                                              batch_size=batch_size,
                                             validation_split =0.2,
                                             subset ='validation',
                                             seed=42,
                                             image_size=(256,256),)

Found 5082 files belonging to 2 classes.
Using 4066 files for training.
Found 5082 files belonging to 2 classes.
Using 1016 files for validation.


In [3]:
#AUTOTUNE = tf.data.AUTOTUNE

In [4]:
#training.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
#val.cache().prefetch(buffer_size=AUTOTUNE)


In [5]:
model = models.Sequential()
model.add(layers.Rescaling(1./255,input_shape=(256,256,3)))
model.add(Conv2D(256,(3,3),padding='Same',activation='relu'))
model.add(Conv2D(256,(3,3),padding='Same',activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(512,(3,3),padding='Same',activation='relu'))
model.add(Conv2D(512,(3,3),padding='Same',activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(1,activation='sigmoid'))



In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 256)     7168      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 256)     590080    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 256)    0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 512)     1180160   
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 512)     2359808   
                                                        

In [9]:
model.compile(optimizer='sgd',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [ ]:
epochs = 10
history = model.fit(training, validation_data=val,epochs=epochs)

Epoch 1/10


2022-05-09 18:57:38.825897: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 28/128 [=====>........................] - ETA: 10:11:57 - loss: 0.6860 - accuracy: 0.5525